# 展開
データの集計結果を表形式に変換する**展開**

In [1]:
import sys
import pandas as pd
sys.path.append("../src")
from data_loader import load_hotel_reserve


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

## 横持ちへの変換
データがレコード形式になっている場合を**縦持ち**、データが表形式の状態の場合を**横持ち**という。

予約テーブルから、顧客/宿泊人数ごとに予約数をカウントし、行を顧客ID、列を宿泊人数、値を予約数の行列に変換する。  
pythonでは横持ち用の関数として利用できる`pivot_table`関数が提供されている。`pivot_table`関数は横持ちに変換するだけでなく、同時に集約処理を行うことができる。

In [3]:
# pivot_table関数で、横持ち変換と集約処理を同時実行
# aggfuncに予約数をカウントする関数を指定
pd.pivot_table(reserve_tb, index='customer_id', columns='people_num',
               values='reserve_id',
               aggfunc=lambda x: len(x), fill_value=0)

people_num,1,2,3,4
customer_id,,,,
c_1,2,2,2,2
c_10,0,2,2,2
c_100,2,1,2,0
c_1000,1,0,0,1
c_101,2,1,1,1
...,...,...,...,...
c_994,1,0,0,0
c_995,2,2,1,3
c_996,0,4,3,0


## スパースマトリックスへの変換
スパースマトリックス：ほとんどの要素の値が0で、ごくわずかしか値が存在しない巨大な行列  
横持ちの場合おいてデータサイズが膨れ上がらないようにするには、縦持ちのデータ表現のまま表にする必要がある。

「横持ちの変換」で作成した行列をスパースマトリックスとして作成する。

In [4]:
from scipy.sparse import csc_matrix

# 顧客ID／宿泊人数別の予約数の表を生成
cnt_tb = reserve_tb \
  .groupby(['customer_id', 'people_num'])['reserve_id'].size() \
  .reset_index()
cnt_tb.columns = ['customer_id', 'people_num', 'rsv_cnt']

# sparseMatrixの行／列に該当する列の値をカテゴリ型に変換
# カテゴリ型については「第9章 カテゴリ型」で詳しく説明
customer_id = pd.Categorical(cnt_tb['customer_id'])
people_num = pd.Categorical(cnt_tb['people_num'])

# スパースマトリックスを生成
# 1の引数は、指定した行列に対応した値、行番号、列番号の配列をまとめたタプルを指定
# shapeには、スパースマトリックスのサイズを指定（行数／列数のタプルを指定）
# （customer_id.codesはインデックス番号の取得）
# （len(customer_id.categories)は、customer_idのユニークな数を取得）
csc_matrix((cnt_tb['rsv_cnt'], (customer_id.codes, people_num.codes)),
           shape=(len(customer_id.categories), len(people_num.categories)))


<888x4 sparse matrix of type '<class 'numpy.int64'>'
	with 2366 stored elements in Compressed Sparse Column format>

scipy.sparseではさまざまなデータ型のスパースマトリックスが提供されている。

- `lil_matrix`：matrixの値を更新するのが速く、演算処理が遅い形式
- `csr_matrix`：行のアクセスが速く、演算処理が速い形式
- `csc_matrix`：列のアクセスが速く、演算処理が速い形式

逐次データを更新していく場合`lil_matrix`を利用、演算処理をする場合には、`csv_matrix`か`csc_matrix`を利用する。`tolil`/`tocsr`/`tocsc`関数で各形式に変換できる。